In [1]:
rm(list=ls())

# Librerías ====
if(!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, showtext, srvyr)

Cargando paquete requerido: pacman



In [8]:
##Crear folders de almacenamiento
dir.create("vivienda/microdatos", showWarnings = F)

In [10]:
# Función para descargar y descomprimir un archivo con timeout
download_and_unzip <- function(url, destination="vivienda/microdatos") {
  tryCatch({
    # Descargar el archivo con timeout
    download.file(url, destfile = paste0(destination, "/", basename(url)), mode = "wb", timeout = 10000)

    # Descomprimir el archivo
    unzip(paste0(destination, "/", basename(url)), exdir = destination)

    # Eliminar el archivo comprimido después de descomprimirlo
    unlink(paste0(destination, "/", basename(url)))

    message(paste("Descarga y descompresión exitosas para", basename(url)))
  }, error = function(e) {
    warning(paste("Error al descargar o descomprimir", basename(url), ":", conditionMessage(e)))
  })
}

In [11]:
download_and_unzip("https://www.inegi.org.mx/contenidos/programas/enigh/nc/2022/microdatos/enigh2022_ns_concentradohogar_csv.zip")
download_and_unzip("https://www.inegi.org.mx/contenidos/programas/enigh/nc/2022/microdatos/enigh2022_ns_viviendas_csv.zip")

Descarga y descompresión exitosas para enigh2022_ns_concentradohogar_csv.zip

Descarga y descompresión exitosas para enigh2022_ns_viviendas_csv.zip



In [50]:
viviendas <- read_csv("vivienda/microdatos/viviendas.csv",show_col_types=FALSE)%>%

mutate(cve_ent = case_when(
      nchar(folioviv) == 9  ~ paste0("0", substr(folioviv, 1, 1)),
      nchar(folioviv) == 10 ~ substr(folioviv, 1, 2)
    )) %>%
    # Filtrar si entidad es 9
  filter(cve_ent == "09")%>%
  filter(tenencia %in% c(3,4))%>%
  mutate(tenenciac= case_when(
    tenencia %in% c (3,4) ~ "Propia",
    TRUE ~ "Otra"),
    forma_adq=case_when(tipo_adqui %in% c(1) ~ "Compra",
                        tipo_adqui %in% c(2) ~ "Manda construir",
                        tipo_adqui %in% c(3) ~ "Autoconstruye",
                        tipo_adqui %in% c(4) ~ "Otra"))%>%
  select(folioviv,tenencia,tipo_adqui,tenenciac,forma_adq)

head(viviendas,10)

Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"


folioviv,tenencia,tipo_adqui,tenenciac,forma_adq
<chr>,<dbl>,<dbl>,<chr>,<chr>
0900006902,4,1,Propia,Compra
0900006903,4,4,Propia,Otra
0900006905,4,1,Propia,Compra
0900009205,4,1,Propia,Compra
0900017403,4,1,Propia,Compra
0900017404,4,1,Propia,Compra
0900126602,4,3,Propia,Autoconstruye
0900126604,4,2,Propia,Manda construir
0900165802,4,1,Propia,Compra


In [70]:
hogares<- read_csv("vivienda/microdatos/concentradohogar.csv",show_col_types=FALSE)%>%
  mutate(cve_ent = case_when(
      nchar(folioviv) == 9  ~ paste0("0", substr(folioviv, 1, 1)),
      nchar(folioviv) == 10 ~ substr(folioviv, 1, 2)
    )) %>%
    # Filtrar si entidad es 9
  filter(cve_ent == "09")%>%
  #Crear deciles
  mutate(decil = ntile(ing_cor, 10))%>%
  #Pegar con viviendas
  left_join(viviendas, by = "folioviv")%>%
    filter(tenenciac == "Propia")

In [71]:
dm<-hogares%>%
as_survey_design(ids = upm, weights = factor,strata=est_dis,
 nest = TRUE)

In [74]:
dm%>%
filter(decil %in%  c(1))%>%
group_by(forma_adq)%>%
summarise(
  hogares=survey_total(vartype="cv"),
  pct=survey_prop( vartype="cv"),
  )%>%
  mutate(pct=round(pct*100,1))%>%
  #Ordenar por pct
  arrange(desc(pct))

forma_adq,hogares,hogares_cv,pct,pct_cv
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Manda construir,30900,0.2596985,37.1,0.1995930
Autoconstruye,28092,0.2322150,33.7,0.2005451
Compra,16293,0.2988470,19.6,0.2781492
Otra,8041,0.4523610,9.7,0.4348759
